In [5]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import OneClassSVM
from sklearn.metrics import confusion_matrix, classification_report, make_scorer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_predict
import numpy as np
import common # type: ignore
from sklearn.feature_selection import VarianceThreshold
from mlxtend.feature_selection import SequentialFeatureSelector
import warnings
import pickle
from pprint import pprint

dataset = common.Dataset()

dataset.label().value_counts()

flags = common.DatasetFlags.ONLY_1M ^ common.DatasetFlags.ONLY_TLD

print(common.DatasetFlags.ONLY_1M in flags)
dataset.feature(flags).columns


True


MultiIndex([('llr', 'TLD', '1000000')],
           )

# DATASET SPLIT

In [8]:
feature, label = dataset.feature(common.DatasetFlags.ONLY_1M ^ common.DatasetFlags.ONLY_TLD), dataset.label()

xy = common.TT()

xy.split(feature, label, test_size=0.8)


(53419, 1)
(213676, 1)
(53419,)
(213676,)


# FEATURE SELECTION

## Variance Threshold

In [25]:
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))

sel.fit(xy.Train.X)

# print(dataset.loc[:, ~sel.get_support(indices=False)].columns)

VarianceThreshold(threshold=0.15999999999999998)

## Forward feature selection

https://www.kdnuggets.com/2018/06/step-forward-feature-selection-python.html

In [10]:
print(xy.Train.X)

rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

sfs = SequentialFeatureSelector(rf, 
                                cv=2, 
                                forward=True, 
                                verbose=2, 
                                k_features=(1, xy.Train.X.shape[1]),
                                scoring="roc_auc")

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    sfs = sfs.fit(xy.Train.X, xy.Train.Y)

filename = 'sfs_cv3_scoringROCAUC.pickle'
pickle.dump(sfs, open(filename, 'wb'))

            llr
            TLD
        1000000
74504     0.000
201286  371.291
193448  282.269
168187    0.000
50330    -3.424
...         ...
213375  271.283
77774   -25.723
168553 -149.837
93011  -183.682
94757  -113.659

[53419 rows x 1 columns]



[2024-06-03 15:27:17] Features: 1/1 -- score: 0.6510853654846677

In [27]:
from sklearn.model_selection import cross_val_score, cross_validate

from common import DatasetFlags

def fpr(y, y_pred):
    false_positives = (y_pred[y == 0] > 0).sum()
    return false_positives / (y == 0).shape[0]

def fp(y, y_pred):
    return (y_pred[y == 0] > 0).sum()
    
def cm(y, y_pred):
    cm = confusion_matrix(y, y_pred)
    return { 'tn': cm[0, 0], 'fp': cm[0, 1], 'fn': cm[1, 0], 'tp': cm[1, 1] }
    
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

skf = StratifiedKFold(n_splits=5, shuffle=True)

scoring = {
    "prec_macro": 'precision_macro',
    "fpr_macro": make_scorer(fpr),
    # "fp": make_scorer(fp),
    # "cm": make_scorer(cm)
}

# predictions = cross_validate(rf, xy.Train.X, xy.Train.Y, cv=skf, scoring=scoring)

predictions = cross_validate(
    rf,
    dataset.feature(DatasetFlags.ONLY_1M ^ DatasetFlags.ONLY_TLD),
    dataset.label(),
    cv=skf,
    scoring=scoring)
    
pprint(predictions)

{'fit_time': array([2.28390718, 2.22627687, 2.39648533, 2.33916998, 2.27246213]),
 'score_time': array([0.0476191 , 0.04666996, 0.04644084, 0.04664612, 0.04551101]),
 'test_fpr_macro': array([0.0165297 , 0.01735338, 0.01660458, 0.01688538, 0.01649226]),
 'test_prec_macro': array([0.65092713, 0.6417917 , 0.65214837, 0.64420913, 0.65447672])}


In [32]:
filename = 'sfs_cv3_scoringROCAUC.pickle'
sfs_loaded = pickle.load(open(filename, 'rb'))

print(sfs_loaded.k_score_)

print(sfs.k_score_)

0.9961343224145401
0.9961343224145401


In [4]:
def fpr(y, y_pred):
    # print("N", (y == 0).sum(), end='\t')
    # print("TN", (y_pred[y == 0] == 0).sum(), end='\t')
    # print("FP", (y_pred[y == 0] > 0).sum(), end='\t')
    # print("FP2", ((y == 0).sum()) - (y_pred[y == 0] > 0).sum(), end='\n')

    false_positives = (y_pred[y == 0] > 0).sum()

    print("FPR", false_positives / (y == 0).sum())
    
    return false_positives / (y == 0).sum()

rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

sfs_fpr = SequentialFeatureSelector(rf, 
                                cv=5, 
                                forward=False, 
                                verbose=2, 
                                k_features=(1, xy.Train.X.shape[1]),
                                scoring=make_scorer(fpr, greater_is_better=True))

# la valutazione viene effettuata alla fine, se riesce o no a trovare i true positive.

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    sfs_fpr = sfs_fpr.fit(xy.Train.X, xy.Train.Y)


filename = 'sfs_cv3_scoringFPR.pickle'
pickle.dump(sfs, open(filename, 'wb'))

FPR 0.14285714285714285
FPR 0.14285714285714285
FPR 0.10204081632653061
FPR 0.14227642276422764
FPR 0.1673469387755102
FPR 0.12244897959183673
FPR 0.13877551020408163
FPR 0.1346938775510204
FPR 0.14634146341463414
FPR 0.17142857142857143
FPR 0.15918367346938775
FPR 0.15918367346938775
FPR 0.11020408163265306
FPR 0.16260162601626016
FPR 0.15918367346938775
FPR 0.1469387755102041
FPR 0.15918367346938775
FPR 0.11428571428571428
FPR 0.16666666666666666
FPR 0.17142857142857143
FPR 0.13877551020408163
FPR 0.15918367346938775
FPR 0.11428571428571428
FPR 0.16260162601626016
FPR 0.17551020408163265
FPR 0.14285714285714285
FPR 0.15510204081632653
FPR 0.11836734693877551
FPR 0.15447154471544716
FPR 0.17142857142857143
FPR 0.1469387755102041
FPR 0.16326530612244897
FPR 0.11428571428571428
FPR 0.14634146341463414
FPR 0.17142857142857143
FPR 0.1469387755102041
FPR 0.15918367346938775
FPR 0.11428571428571428
FPR 0.16666666666666666
FPR 0.1673469387755102
FPR 0.16326530612244897
FPR 0.1673469387755102

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:  1.8min


FPR 0.11836734693877551
FPR 0.1510204081632653
FPR 0.11020408163265306
FPR 0.16666666666666666
FPR 0.17142857142857143
FPR 0.14285714285714285
FPR 0.1510204081632653
FPR 0.12653061224489795
FPR 0.15447154471544716
FPR 0.17142857142857143
FPR 0.1469387755102041
FPR 0.1510204081632653
FPR 0.11020408163265306
FPR 0.14227642276422764
FPR 0.17142857142857143
FPR 0.14285714285714285
FPR 0.16326530612244897
FPR 0.12244897959183673
FPR 0.15853658536585366
FPR 0.1673469387755102
FPR 0.1346938775510204
FPR 0.13877551020408163
FPR 0.11428571428571428
FPR 0.13821138211382114
FPR 0.16326530612244897
FPR 0.13877551020408163
FPR 0.14285714285714285
FPR 0.11020408163265306
FPR 0.15853658536585366
FPR 0.16326530612244897
FPR 0.13877551020408163
FPR 0.1510204081632653
FPR 0.11020408163265306
FPR 0.15040650406504066
FPR 0.17142857142857143
FPR 0.1306122448979592
FPR 0.1510204081632653
FPR 0.12653061224489795
FPR 0.14634146341463414
FPR 0.15918367346938775
FPR 0.22040816326530613
FPR 0.1836734693877551
FP


[2024-06-03 11:36:10] Features: 53/1 -- score: 0.20960013273602124

FPR 0.23265306122448978
FPR 0.2
FPR 0.15918367346938775
FPR 0.23577235772357724
FPR 0.23673469387755103
FPR 0.21224489795918366
FPR 0.21224489795918366
FPR 0.16326530612244897
FPR 0.24796747967479674
FPR 0.22040816326530613
FPR 0.20408163265306123
FPR 0.19591836734693877
FPR 0.1836734693877551
FPR 0.23170731707317074
FPR 0.22448979591836735
FPR 0.2163265306122449
FPR 0.19591836734693877
FPR 0.1836734693877551
FPR 0.26422764227642276
FPR 0.23265306122448978
FPR 0.21224489795918366
FPR 0.2
FPR 0.17551020408163265
FPR 0.23170731707317074
FPR 0.23265306122448978
FPR 0.2163265306122449
FPR 0.20816326530612245
FPR 0.16326530612244897
FPR 0.24796747967479674
FPR 0.24897959183673468
FPR 0.20408163265306123
FPR 0.21224489795918366
FPR 0.15918367346938775
FPR 0.24390243902439024
FPR 0.23265306122448978
FPR 0.24489795918367346
FPR 0.22448979591836735
FPR 0.2
FPR 0.2804878048780488
FPR 0.27346938775510204
FPR 0.2163265306122449
FPR 0.20408163265306123
FPR 0.16326530612244897
FPR 0.2479674796747967

...


[2024-06-03 12:19:35] Features: 3/1 -- score: 1.0

FPR 1.0
FPR 1.0
FPR 1.0
FPR 1.0
FPR 1.0
FPR 1.0
FPR 1.0
FPR 1.0
FPR 1.0
FPR 1.0
FPR 1.0
FPR 1.0
FPR 1.0
FPR 1.0
FPR 1.0
FPR 1.0



[2024-06-03 12:19:38] Features: 2/1 -- score: 1.0

FPR 1.0
FPR 1.0
FPR 1.0
FPR 1.0
FPR 1.0
FPR 1.0
FPR 1.0
FPR 1.0
FPR 1.0



[2024-06-03 12:19:39] Features: 1/1 -- score: 1.0

In [22]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, class_weight={ 0: 10, 2: 1})

sfs_fpr = SequentialFeatureSelector(rf, 
                                cv=3, 
                                forward=True, 
                                verbose=2, 
                                k_features=(1, xy.Train.X.shape[1]),
                                scoring="roc_auc")

# la valutazione viene effettuata alla fine, se riesce o no a trovare i true positive.

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    sfs_weight10 = sfs_fpr.fit(xy.Train.X, xy.Train.Y)


[2024-06-01 17:46:20] Features: 1/34 -- score: 0.9222458407289357
[2024-06-01 17:46:34] Features: 2/34 -- score: 0.9653776171221398
[2024-06-01 17:46:48] Features: 3/34 -- score: 0.972064448632667
[2024-06-01 17:47:02] Features: 4/34 -- score: 0.9741350791453135
[2024-06-01 17:47:16] Features: 5/34 -- score: 0.973031679065441
[2024-06-01 17:47:30] Features: 6/34 -- score: 0.9738717843519229
[2024-06-01 17:47:42] Features: 7/34 -- score: 0.9735330190579159
[2024-06-01 17:47:55] Features: 8/34 -- score: 0.9731140614877528
[2024-06-01 17:48:07] Features: 9/34 -- score: 0.9730544034086281
[2024-06-01 17:48:19] Features: 10/34 -- score: 0.9726457151053426
[2024-06-01 17:48:30] Features: 11/34 -- score: 0.974105859539511
[2024-06-01 17:48:41] Features: 12/34 -- score: 0.9734194214655228
[2024-06-01 17:48:51] Features: 13/34 -- score: 0.986544131780588
[2024-06-01 17:49:02] Features: 14/34 -- score: 0.9928287234052809
[2024-06-01 17:49:14] Features: 15/34 -- score: 0.9936939191892343
[2024-0

In [14]:
sfs__ = sfs

print(sfs__, sfs)

SequentialFeatureSelector(cv=3, estimator=RandomForestClassifier(n_jobs=-1),
                          k_features=(1, 34), scoring='roc_auc', verbose=2) SequentialFeatureSelector(cv=3, estimator=RandomForestClassifier(n_jobs=-1),
                          k_features=(1, 34), scoring='roc_auc', verbose=2)


In [20]:

sfs.k_feature_idx_

feature = dataset.feature()
feature[feature.columns[list(sfs.k_feature_idx_)]]

feature_selected = []
for i in range(34):
    if i not in list(sfs.k_feature_idx_):
        print(feature.columns[i])
    else:
        feature_selected.append(feature.columns[i])

('-', '-', 'qr')
('eps', 'NONE', '[0.100, 0.250)')
('eps', 'NONE', '[0.500, 0.900)')
('eps', 'TLD', '[0.000, 0.100)')
('eps', 'TLD', '[0.900, 1.100)')
('eps', 'ICANN', '[0.000, 0.100)')
('eps', 'PRIVATE', '[0.000, 0.100)')


In [ ]:
def fpr(y, y_pred):
    false_positives = (y_pred[y == 0] > 0).sum()
    return false_positives / (y == 0).shape[0]

def fp(y, y_pred):
    return (y_pred[y == 0] > 0).sum()
    
def cm(y, y_pred):
    cm = confusion_matrix(y, y_pred)
    return {'tn': cm[0, 0], 'fp': cm[0, 1],
             'fn': cm[1, 0], 'tp': cm[1, 1]}
    
rf = RandomForestClassifier()

dataset, label = common.load_dataset()

dataset = dataset.iloc[0:10000]
label = label.iloc[0:10000]

skf = StratifiedKFold(n_splits=5)

scoring = {'prec_macro': 'precision_macro',
           'rec_macro': make_scorer(fpr),
            "fp": make_scorer(fp),
           "cm": make_scorer(cm)
          }

predictions = cross_val_predict(rf, dataset, label, cv=skf)

print(predictions)

[2 2 2 ... 0 0 0]


In [ ]:
len(predictions)

10000

In [ ]:
dataset, label = common.load_dataset()

for i in range(5):
    _labels = label.iloc[scores['indices']['train'][i]]
    print((_labels == 0).sum(), (_labels > 0).sum())
    _labels = label.iloc[scores['indices']['test'][i]]
    print((_labels == 0).sum(), (_labels > 0).sum())
    print(scores["train_rec_macro"][i])
    print(scores["train_fp"][i])
    print(scores["test_rec_macro"][i])
    print(scores["test_fp"][i])
    print()

3774 4226
943 1057
0.0
0
0.0995
199

3774 4226
943 1057
0.0
0
0.044
88

3774 4226
943 1057
0.0
0
0.0
0

3773 4227
944 1056
0.0
0
0.0
0

3773 4227
944 1056
0.0
0
0.0015
3



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8)
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))

RandomForestClassifier()

In [ ]:

x_cols = [
#     (  '-',           '-',      'n_message'),
#     (  '-',           '-',     'fn_req_min'),
#     (  '-',           '-',     'fn_req_max'),
#     (  '-',           '-',   'time_s_start'),
#     (  '-',           '-',     'time_s_end'),
#     (  '-',           '-',     'duration'),
#     (  '-',           '-',             'qr'),
#     (  '-',           '-',              'u'),
#     (  '-',           '-',              'q'),
#     (  '-',           '-',              'r'),
#     (  '-',           '-',             'nx'),
#     (  '-', 'whitelisted',              '0'),
#     (  '-', 'whitelisted',             '100'),
#     (  '-', 'whitelisted',             '1000'),
#     (  '-', 'whitelisted',             '1000000'),
#     ('eps',        'NONE', '[0.000, 0.100)'),
#     ('eps',        'NONE', '[0.100, 0.250)'),
#     ('eps',        'NONE', '[0.250, 0.500)'),
#     ('eps',        'NONE', '[0.500, 0.900)'),
#     ('eps',        'NONE', '[0.900, 1.100)'),
    ('eps',         'TLD', '[0.000, 0.100)'),
    ('eps',         'TLD', '[0.100, 0.250)'),
    ('eps',         'TLD', '[0.250, 0.500)'),
    ('eps',         'TLD', '[0.500, 0.900)'),
    ('eps',         'TLD', '[0.900, 1.100)'),
#     ('eps',       'ICANN', '[0.000, 0.100)'),
#     ('eps',       'ICANN', '[0.100, 0.250)'),
#     ('eps',       'ICANN', '[0.250, 0.500)'),
#     ('eps',       'ICANN', '[0.500, 0.900)'),
#     ('eps',       'ICANN', '[0.900, 1.100)'),
#     ('eps',     'PRIVATE', '[0.000, 0.100)'),
#     ('eps',     'PRIVATE', '[0.100, 0.250)'),
#     ('eps',     'PRIVATE', '[0.250, 0.500)'),
#     ('eps',     'PRIVATE', '[0.500, 0.900)'),
#     ('eps',     'PRIVATE', '[0.900, 1.100)'),
#     ('llr',        'ninf',           'NONE'),
#     ('llr',        'ninf',            'TLD'),
#     ('llr',        'ninf',          'ICANN'),
#     ('llr',        'ninf',        'PRIVATE'),
#     ('llr',        'pinf',           'NONE'),
#     ('llr',        'pinf',            'TLD'),
#     ('llr',        'pinf',          'ICANN'),
#     ('llr',        'pinf',        'PRIVATE'),
#     ('llr',        'NONE',              '0'),
#     ('llr',        'NONE',             '100'),
#     ('llr',        'NONE',             '1000'),
#     ('llr',        'NONE',             '1000000'),
#     ('llr',         'TLD',              '0'),
#     ('llr',         'TLD',             '100'),
#     ('llr',         'TLD',             '1000'),
#     ('llr',         'TLD',             '1000000'),
#     ('llr',       'ICANN',              '0'),
#     ('llr',       'ICANN',             '100'),
#     ('llr',       'ICANN',             '1000'),
#     ('llr',       'ICANN',             '1000000'),
#     ('llr',     'PRIVATE',              '0'),
#     ('llr',     'PRIVATE',             '100'),
#     ('llr',     'PRIVATE',             '1000'),
#     ('llr',     'PRIVATE',             '1000000')
]

label_col = ('-', 'source', 'mc')
dataset_cols = x_cols + [label_col]

dataset_cols

[('eps', 'TLD', '[0.000, 0.100)'),
 ('eps', 'TLD', '[0.100, 0.250)'),
 ('eps', 'TLD', '[0.250, 0.500)'),
 ('eps', 'TLD', '[0.500, 0.900)'),
 ('eps', 'TLD', '[0.900, 1.100)'),
 ('-', 'source', 'mc')]

In [ ]:
oc = OneClassSVM(nu=0.00005)

X_train = df[dataset_cols].copy()
X_test = df[dataset_cols].copy()

X_train = X_train[X_train[label_col] == 0]
X_train, X_0_test = train_test_split(X_train, test_size=0.5)

X_test = X_test[X_test[label_col] > 0]
X_test = pd.concat([X_test, X_0_test], axis=0)
y_test_true = X_test[label_col].apply(lambda x: 1 if x == 0 else -1)

y_train_true = np.ones(X_train.shape[0])

X_train = X_train.drop(columns=[label_col])
X_test = X_test.drop(columns=[label_col])

y_train_pred = oc.fit_predict(X_train)
# y_train_pred = oc.predict(X_train)
y_test_pred = oc.predict(X_test)

# print(classification_report(y_train_true, y_train_pred))
# print(classification_report(y_test_true, y_test_pred))

print(confusion_matrix(y_train_true, y_train_pred, labels=[1,-1]))
print(confusion_matrix(y_test_true, y_test_pred, labels=[1,-1]))

[[2981    8]
 [   0    0]]
[[  2987      3]
 [131478 129638]]


In [ ]:

X_train[y_train_pred == 1]


-                    llr
             -                    TLD
      duration   u   q   r nx      50
3608    10.306  16  50  50  0 -97.643
7406     4.437  23  50  50  0 -36.746
577      8.747  15  50  50  0 -38.051
3835    12.550  21  50  49  0 -72.370
8060    10.157  27  50  50  0 -51.730
...        ...  ..  ..  .. ..     ...
9596    18.669  22  50  50  0 -68.456
5946    20.557  25  50  50  0 -34.554
10693    0.000  24  50  46  0 -63.290
11162    0.000  25  50  35  0 -70.668
11228    0.000  28  50  52  0 -80.603

[596 rows x 6 columns]

In [ ]:

oc = OneClassSVM()

# X_train = X_train[y_train_pred == 1]
y_train_true = np.ones(X_train[y_train_pred == 1].shape[0])

y_train_pred = oc.fit_predict(X_train[y_train_pred == 1])

print(confusion_matrix(y_train_true, y_train_pred, labels=[1,-1]))

[[298 298]
 [  0   0]]


In [ ]:
X = df[labels == 0]
X

l1         -                                                       
          l2    source                                                       
          l3 manyindex        galaxy                                  name   
31        31         2  stratosphere  12_2013-12-17_capture1.only-dns.pcap  \
32        32         2  stratosphere  12_2013-12-17_capture1.only-dns.pcap   
33        33         2  stratosphere  12_2013-12-17_capture1.only-dns.pcap   
34        34         2  stratosphere  12_2013-12-17_capture1.only-dns.pcap   
35        35         2  stratosphere  12_2013-12-17_capture1.only-dns.pcap   
...      ...       ...           ...                                   ...   
11257  11257        32  stratosphere        22_2017-05-02_kali-normal.pcap   
11258  11258        32  stratosphere        22_2017-05-02_kali-normal.pcap   
11259  11259        32  stratosphere        22_2017-05-02_kali-normal.pcap   
11260  11260        32  stratosphere        22_2017-05-02_kali-normal.pcap   
11261  11261        32  stratosphere        22_2017-05-02_kali-normal.pcap   

                                            ...       llr                       
                    -                       ...       TLD               ICANN   
       id bc mc index n_message fn_req_min  ...        15        50         0   
31      9  0  0     0       320          0  ... -1405.590 -1405.590  -845.654  \
32      9  0  0     1       336         50  ... -1461.434 -1461.434  -898.159   
33      9  0  0     2       384        100  ... -1661.061 -1661.061 -1013.272   
34      9  0  0     3       376        150  ... -1590.254 -1590.254  -988.605   
35      9  0  0     4       384        200  ... -1658.997 -1658.997 -1009.987   
...    .. .. ..   ...       ...        ...  ...       ...       ...       ...   
11257  13  0  0  1036       152      51800  ...     0.000     0.000  -230.909   
11258  13  0  0  1037       104      51850  ...   -16.611   -16.611  -126.462   
11259  13  0  0  1038       176      51900  ...   -59.571   -59.571  -225.193   
11260  13  0  0  1039       216      51950  ...   -22.778   -22.778  -325.789   
11261  13  0  0  1040       144      52000  ...   -34.062   -34.062  -218.858   

                                                                         
                                     PRIVATE                             
             10        15        50        0       10       15       50  
31     -845.654  -845.654  -845.654 -557.478 -557.478 -557.478 -557.478  
32     -898.159  -898.159  -898.159 -580.093 -580.093 -580.093 -580.093  
33    -1013.272 -1013.272 -1013.272 -665.097 -665.097 -665.097 -665.097  
34     -967.213  -967.213  -967.213 -662.137 -649.203 -649.203 -649.203  
35    -1009.987 -1009.987 -1009.987 -652.982 -652.982 -652.982 -652.982  
...         ...       ...       ...      ...      ...      ...      ...  
11257     0.000     0.000     0.000 -203.955    0.000    0.000    0.000  
11258    14.333    14.333    14.333 -111.523   14.151   14.151   14.151  
11259   -12.190   -12.190   -12.190 -169.303   28.537   28.537   28.537  
11260   -14.694   -14.694   -14.694 -322.485  -11.343  -11.343  -11.343  
11261   -30.221   -30.221   -30.221 -211.444  -31.820  -31.820  -31.820  

[5979 rows x 66 columns]